In [1]:
from model_persistant_state import ModelPersistantState
from dataset import get_dataloader
import torch
import numpy as np

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. load a model
model_path = "models/mamba2"
results_path = "results/abx/"
model_persistant_state = ModelPersistantState(model_path)

In [3]:
model = model_persistant_state.load_model()

In [4]:
# 2. load a dataset
dataloader = get_dataloader(20, data_type="abx")

In [5]:
for idx, (waveforms, lens, path) in enumerate(dataloader):
    model.eval()
    with torch.no_grad():
        result = model(waveforms, lens, only_student=True)
        result = result["student"]
    paths = [results_path + ".".join("/".join(p.split("/")[-2:]).split(".")[:-1]) + ".npy" for p in path]
    for batch_idx, path in enumerate(paths):
        with open(path, "w"):
            # write tensor to the npy file in numpy format
            result_to_save = result[batch_idx].cpu().numpy()
            # transform to float64
            result_to_save = result_to_save.astype(np.float64)[:lens[batch_idx]]
            np.save(path, result_to_save)
            # torch.save(result[i], path)
    # print the progress in precentage
    print(f"\r{idx}/{len(dataloader)}", end="")